#### Loading weather data into PostgreSQL
*Author: Blake Bentley*

In [1]:
# Prepare environment
import pandas as pd
import os 
import matplotlib.pyplot as plt
import numpy as np 
import psycopg2 as pg
import io
from sqlalchemy import create_engine

# Set working directory
os.chdir("C:/Users/Blake/Documents/UTS/36102_iLab_1/Client/Climate/")
# You will need subdirectories below this called 'Historical' and 'Historical/Data'
# Put the reference data in Historical. Put the text files containing the data into Historical/Data

# Define this data path
fnpath = '''Historical/Data/'''

In [ ]:
# Define input parameters for connecting to the database
user_name = r"postgres"
user_pass = r"password"
db_name = r"endgame"
server = r"localhost"

# Connection string
strEngine = r'postgresql://' + user_name+ "@" + server + "/" + db_name

# Define for weather data
loadSchema = "bom"
# Define table name for station reference
tblStationRef = "station_ref"
# Define table name for weather data
tblWeather = "weather"

#### The stations definItion file
Begin by loading the station reference table. This table provides meta information on each weather station. 

In [ ]:
# Import the stations definition file using the following headers
statdefNames = ['st', 'station_id', 'rain_dist_c', 'station_name', 'open_d', 'clse_d'
, 'latd', 'lgtd', 'coord_src', 'state', 'elev', 'barom_elev', 'wmo_id', 'efft_d', 'expy_d'
, 'pc_complete', 'pc_qual_y', 'pc_qual_n', 'pc_qual_w', 'pc_qual_s', 'pc_qual_i', 'symbol']

# Read the file in
statdef = pd.read_csv("Historical/HM01X_StnDet_999999999395130.txt"
                      , header = None
                      , names = statdefNames
                      , index_col = 'station_id')
statdef.head()
del statdefNames

In [ ]:
# Clean up the table
# 1. Drop the redundant fields
statdefDel = ['st', 'symbol', 'elev', 'barom_elev', 'wmo_id', 'coord_src']
statdef = statdef.drop(statdefDel, axis = 1)
statdef.head()
del statdefDel

# 2. Strip white space from name
statdef['station_name'] = statdef['station_name'].str.strip()
statdef.head()

In [ ]:
# Load the station reference table into postgres
# Define a connection string 
engine = create_engine(strEngine)

# Write the table
statdef.to_sql(tblStationRef, engine, schema = loadSchema)

# Check the table exists 
print (engine.has_table(tblStationRef, schema = loadSchema))

#### Load the weather collected for each station
Each weather station has a file containing timestamped measures of climate. The following steps will read these files into python, cleanse and prepare them for load into a PostgreSQL database. 

In [ ]:
# Define the column headers as a list of names
weatherName = ['rec_id', 'station_id', 'lcl_yr', 'lcl_mnth', 'lcl_day', 'lcl_hr', 'lcl_min'
,'std_yr', 'std_mnth', 'std_day', 'std_hr', 'std_min', 'air_temp', 'qual_flag', 'aws', 'symbol']

# Define the types to load the data as
weatherType = {'rec_id':np.object, 'station_id':np.object, 'lcl_yr':np.int32, 'lcl_mnth':np.int32
, 'lcl_day':np.int32, 'lcl_hr':np.int32, 'lcl_min':np.int32, 'std_yr':np.int32, 'std_mnth':np.int32
, 'std_day':np.int32, 'std_hr':np.int32, 'std_min':np.int32, 'air_temp':np.object, 'qual_flag':np.object
, 'aws':np.object, 'symbol':np.object}

In [ ]:
# Now begin the loop

# Begin the loop
for fn in os.listdir(fnpath):
    print (os.path.join(fnpath, fn))
    
    # Read in the file
    fnname = os.path.join(fnpath, fn)
    thisfile = pd.read_csv(fnname, skiprows=1, names=weatherName, dtype=weatherType, na_values=" ")
    
    # Drop some cols
    thisfileDel = ['lcl_yr', 'lcl_mnth', 'lcl_day', 'lcl_hr', 'lcl_min', 'aws', 'symbol', 'rec_id']
    thisfile = thisfile.drop(thisfileDel, axis = 1)
    
    # Create a datetime column
    thisfile.index = pd.to_datetime(dict(year = thisfile.std_yr
                                         , month = thisfile.std_mnth
                                         , day = thisfile.std_day
                                         , hour = thisfile.std_hr
                                         , minute = thisfile.std_min))
    
    # Drop some more cols
    thisfileDel = ['std_yr', 'std_mnth', 'std_day', 'std_hr', 'std_min']
    thisfile = thisfile.drop(thisfileDel, axis = 1)
    
    # Strip the whitespace from the air temp and station ID column
    thisfile['air_temp'] = thisfile['air_temp'].str.strip()
    thisfile['station_id'] = thisfile['station_id'].str.strip()
    
    # Then change the column type
    thisfile['air_temp'] = pd.to_numeric(thisfile['air_temp'], errors='coerce')
    
    #thisfile.head()
    #thisfile.info()
    
    # With the data set clean, load it into the database
    thisfile.to_sql(tblWeather, engine, schema = loadSchema, if_exists='append')


In [ ]:
# Finish by putting an index on the station ID field in the database
engine.execute("create index idx_station_weather on " + loadSchema + "." + tblWeather + "using btree (station_id);")

In [2]:
os.system('jupyter nbconvert --to html LoadClimate.ipynb')

0